# 인터랙티브 대시보드 드랍다운 테스트 노트북

이 노트북은 제조 데이터 이상치 탐지 시스템의 인터랙티브 대시보드에서 eqp, chamber, recipe 드랍다운이 정상적으로 출력되고 동작하는지 확인합니다.

## 목표
- ipywidgets 드랍다운이 Jupyter 환경에서 정상적으로 표시되는지 확인
- 드랍다운 선택에 따른 데이터 필터링 및 시각화 동작 확인
- 환경별(Jupyter/VS Code/터미널) 호환성 테스트
- 문제 발생 시 대안 인터페이스 제공

## 1. 필요 라이브러리 임포트 및 환경 확인

먼저 필요한 라이브러리들을 임포트하고 현재 실행 환경을 확인합니다.

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# ipywidgets 임포트 및 확인
try:
    import ipywidgets as widgets
    from ipywidgets import interact, interactive, fixed, interact_manual
    from IPython.display import display, clear_output, HTML
    print("✅ ipywidgets 라이브러리가 성공적으로 로드되었습니다.")
    WIDGETS_AVAILABLE = True
except ImportError as e:
    print(f"❌ ipywidgets 임포트 실패: {e}")
    print("pip install ipywidgets 명령어로 설치하세요.")
    WIDGETS_AVAILABLE = False

✅ ipywidgets 라이브러리가 성공적으로 로드되었습니다.


In [2]:
# 현재 실행 환경 감지
def detect_environment():
    """현재 실행 환경을 감지하는 함수"""
    try:
        # Jupyter 환경 확인
        from IPython import get_ipython
        if get_ipython() is not None:
            ipython = get_ipython()
            if hasattr(ipython, 'kernel'):
                return "jupyter"
    except ImportError:
        pass
    
    # VS Code 환경 확인
    import os
    if 'VSCODE_PID' in os.environ or 'TERM_PROGRAM' in os.environ:
        return "vscode"
    
    return "terminal"

# 환경 정보 출력
environment = detect_environment()
print(f"🔍 감지된 환경: {environment}")

# Jupyter 커널 정보 확인
try:
    from IPython import get_ipython
    ipython = get_ipython()
    if ipython:
        print(f"📝 IPython 버전: {ipython}")
        print(f"🔧 커널 정보: {hasattr(ipython, 'kernel')}")
        
        # 위젯 백엔드 확인
        try:
            import jupyter_widgets
            print(f"✅ jupyter_widgets 버전: {jupyter_widgets.__version__}")
        except ImportError:
            print("❌ jupyter_widgets가 설치되지 않았습니다.")
            
except Exception as e:
    print(f"⚠️ IPython 정보 확인 실패: {e}")

print(f"\n📊 환경별 위젯 지원:")
print(f"  - Jupyter Notebook/Lab: {'✅ 지원' if environment == 'jupyter' else '❌ 미지원'}")
print(f"  - VS Code: {'⚠️ 부분 지원' if environment == 'vscode' else '❌ 미지원'}")
print(f"  - 터미널: {'❌ 미지원' if environment == 'terminal' else '❌ 미지원'}")

🔍 감지된 환경: jupyter
📝 IPython 버전: <ipykernel.zmqshell.ZMQInteractiveShell object at 0x000001DAC0ADF320>
🔧 커널 정보: True
❌ jupyter_widgets가 설치되지 않았습니다.

📊 환경별 위젯 지원:
  - Jupyter Notebook/Lab: ✅ 지원
  - VS Code: ❌ 미지원
  - 터미널: ❌ 미지원


## 2. 데이터 로드 및 전처리

제조 데이터를 로드하고 드랍다운에 사용할 고유값들을 추출합니다.

In [2]:
# 데이터 로드
try:
    df = pd.read_csv('sample_manufacturing.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    print(f"✅ 데이터 로드 성공: {len(df):,}개 행")
    
    # 데이터 미리보기
    print(f"\n📋 데이터 컬럼: {list(df.columns)}")
    print(f"📅 시간 범위: {df['timestamp'].min()} ~ {df['timestamp'].max()}")
    
except FileNotFoundError:
    print("❌ sample_manufacturing.csv 파일을 찾을 수 없습니다.")
    print("샘플 데이터를 생성합니다...")
    
    # 샘플 데이터 생성
    np.random.seed(42)
    n_samples = 1000
    
    timestamps = pd.date_range('2024-01-01', periods=n_samples, freq='5min')
    equipment_ids = ['EQP001', 'EQP002', 'EQP003']
    chamber_ids = ['A', 'B', 'C']
    recipes = ['R101', 'R102', 'R103']
    
    data = []
    for i, ts in enumerate(timestamps):
        eqp_id = np.random.choice(equipment_ids)
        chamber_id = np.random.choice(chamber_ids)
        recipe = np.random.choice(recipes)
        
        # 정상 범위의 센서 데이터
        temp = np.random.normal(400, 20)
        pressure = np.random.normal(1.5, 0.3)
        rf_power = np.random.normal(250, 30)
        chamber_load = np.random.normal(0.8, 0.1)
        
        # 이상치 주입 (5% 확률)
        if np.random.random() < 0.05:
            temp += np.random.normal(100, 20)  # 온도 이상치
        if np.random.random() < 0.03:
            rf_power += np.random.normal(100, 30)  # RF 파워 이상치
        
        data.append({
            'timestamp': ts,
            'eqp_id': eqp_id,
            'chamber_id': chamber_id,
            'recipe': recipe,
            'TEMP': temp,
            'PRESSURE': pressure,
            'RF_POWER': rf_power,
            'CHAMBER_LOAD': chamber_load
        })
    
    df = pd.DataFrame(data)
    df.to_csv('sample_manufacturing.csv', index=False)
    print(f"✅ 샘플 데이터 생성 완료: {len(df):,}개 행")

# 고유값 추출
equipment_ids = ['All'] + sorted(df['eqp_id'].unique().tolist())
chamber_ids = ['All'] + sorted(df['chamber_id'].unique().tolist())
recipes = ['All'] + sorted(df['recipe'].unique().tolist())

print(f"\n🎛️ 드랍다운 옵션:")
print(f"  Equipment IDs: {equipment_ids}")
print(f"  Chamber IDs: {chamber_ids}")
print(f"  Recipes: {recipes}")

# 데이터 품질 확인
print(f"\n📊 데이터 품질:")
print(f"  결측값: {df.isnull().sum().sum()}개")
print(f"  중복 행: {df.duplicated().sum()}개")
print(f"  온도 범위: {df['TEMP'].min():.1f} ~ {df['TEMP'].max():.1f}°C")
print(f"  RF 파워 범위: {df['RF_POWER'].min():.1f} ~ {df['RF_POWER'].max():.1f}")

# 이상치 확인
temp_anomalies = df[df['TEMP'] > 450]
power_anomalies = df[df['RF_POWER'] > 300]
print(f"  온도 이상치: {len(temp_anomalies)}개")
print(f"  RF 파워 이상치: {len(power_anomalies)}개")

✅ 데이터 로드 성공: 7,776개 행

📋 데이터 컬럼: ['timestamp', 'eqp_id', 'chamber_id', 'recipe', 'MPH', 'TEMP', 'PRESSURE', 'RF_POWER', 'GAS_FLOW_1', 'GAS_FLOW_2', 'CHAMBER_LOAD']
📅 시간 범위: 2025-06-24 00:00:00 ~ 2025-06-26 23:55:00

🎛️ 드랍다운 옵션:
  Equipment IDs: ['All', 'EQP001', 'EQP002', 'EQP003']
  Chamber IDs: ['All', 'A', 'B', 'C']
  Recipes: ['All', 'R101', 'R102', 'R103']

📊 데이터 품질:
  결측값: 0개
  중복 행: 0개
  온도 범위: 417.6 ~ 488.1°C
  RF 파워 범위: 212.0 ~ 367.8
  온도 이상치: 77개
  RF 파워 이상치: 76개


## 3. 드랍다운 위젯 생성 및 표시

ipywidgets를 사용해 드랍다운 위젯을 생성하고 화면에 표시합니다.

In [ ]:
# 기본 드랍다운 위젯 생성
if WIDGETS_AVAILABLE:
    print("🎛️ 드랍다운 위젯 생성 중...")
    
    # 개별 드랍다운 위젯 생성
    equipment_dropdown = widgets.Dropdown(
        options=equipment_ids,
        value='All',
        description='Equipment:',
        style={'description_width': '100px'},
        layout={'width': '200px'}
    )
    
    chamber_dropdown = widgets.Dropdown(
        options=chamber_ids,
        value='All',
        description='Chamber:',
        style={'description_width': '100px'},
        layout={'width': '200px'}
    )
    
    recipe_dropdown = widgets.Dropdown(
        options=recipes,
        value='All',
        description='Recipe:',
        style={'description_width': '100px'},
        layout={'width': '200px'}
    )
    
    print("✅ 드랍다운 위젯 생성 완료")
    
    # 위젯 개별 표시 테스트
    print("\n1️⃣ 개별 위젯 표시 테스트:")
    print("Equipment 드랍다운:")
    display(equipment_dropdown)
    
else:
    print("❌ ipywidgets를 사용할 수 없습니다.")

🎛️ 드랍다운 위젯 생성 중...
✅ 드랍다운 위젯 생성 완료

1️⃣ 개별 위젯 표시 테스트:
Equipment 드랍다운:


Dropdown(description='Equipment:', layout=Layout(width='200px'), options=('All', 'EQP001', 'EQP002', 'EQP003')…

In [4]:
# 모든 드랍다운을 함께 표시
if WIDGETS_AVAILABLE:
    print("2️⃣ 모든 드랍다운 함께 표시:")
    
    # HBox 레이아웃으로 가로 배치
    widget_box = widgets.HBox([
        equipment_dropdown, 
        chamber_dropdown, 
        recipe_dropdown
    ])
    
    # VBox 레이아웃으로 세로 배치 (대안)
    widget_vbox = widgets.VBox([
        widgets.HTML("<b>필터 선택:</b>"),
        equipment_dropdown,
        chamber_dropdown,
        recipe_dropdown
    ])
    
    print("가로 배치 (HBox):")
    display(widget_box)
    
    print("\n세로 배치 (VBox):")
    display(widget_vbox)
    
    # 위젯 값 확인
    print(f"\n현재 선택된 값:")
    print(f"  Equipment: {equipment_dropdown.value}")
    print(f"  Chamber: {chamber_dropdown.value}")
    print(f"  Recipe: {recipe_dropdown.value}")

else:
    print("❌ ipywidgets를 사용할 수 없으므로 수동 입력을 사용하세요.")
    
    # 수동 입력 대안
    print("\n📝 수동 입력 방식:")
    equipment_choice = input(f"Equipment 선택 {equipment_ids} [기본값: All]: ") or "All"
    chamber_choice = input(f"Chamber 선택 {chamber_ids} [기본값: All]: ") or "All"
    recipe_choice = input(f"Recipe 선택 {recipes} [기본값: All]: ") or "All"
    
    print(f"\n선택된 값:")
    print(f"  Equipment: {equipment_choice}")
    print(f"  Chamber: {chamber_choice}")
    print(f"  Recipe: {recipe_choice}")

2️⃣ 모든 드랍다운 함께 표시:
가로 배치 (HBox):



세로 배치 (VBox):



현재 선택된 값:
  Equipment: All
  Chamber: All
  Recipe: All


## 4. 드랍다운 선택에 따른 데이터 필터링 및 Plotly 시각화

드랍다운 값이 변경될 때마다 데이터를 필터링하고 Plotly 그래프로 시각화하는 인터랙티브 함수를 작성합니다.

In [5]:
# 데이터 필터링 및 시각화 함수
def create_filtered_visualization(equipment_id='All', chamber_id='All', recipe='All'):
    """선택된 필터에 따라 데이터를 필터링하고 시각화"""
    
    # 데이터 필터링
    filtered_df = df.copy()
    filter_conditions = []
    
    if equipment_id != 'All':
        filtered_df = filtered_df[filtered_df['eqp_id'] == equipment_id]
        filter_conditions.append(f"Equipment: {equipment_id}")
    if chamber_id != 'All':
        filtered_df = filtered_df[filtered_df['chamber_id'] == chamber_id]
        filter_conditions.append(f"Chamber: {chamber_id}")
    if recipe != 'All':
        filtered_df = filtered_df[filtered_df['recipe'] == recipe]
        filter_conditions.append(f"Recipe: {recipe}")
    
    filter_str = " & ".join(filter_conditions) if filter_conditions else "전체 데이터"
    
    # 데이터가 없는 경우 처리
    if len(filtered_df) == 0:
        print(f"⚠️ 선택된 조건 ({filter_str})에 해당하는 데이터가 없습니다.")
        return
    
    # 서브플롯 생성
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            f'Temperature Over Time ({filter_str})',
            f'RF Power Over Time ({filter_str})',
            f'Temperature vs RF Power',
            f'Data Summary'
        ],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"type": "pie"}]]
    )
    
    # 1. 온도 시계열
    fig.add_trace(
        go.Scatter(
            x=filtered_df['timestamp'], 
            y=filtered_df['TEMP'],
            mode='lines', 
            name='Temperature',
            line=dict(color='red', width=2)
        ),
        row=1, col=1
    )
    
    # 온도 이상치 표시
    temp_anomalies = filtered_df[filtered_df['TEMP'] > 450]
    if len(temp_anomalies) > 0:
        fig.add_trace(
            go.Scatter(
                x=temp_anomalies['timestamp'], 
                y=temp_anomalies['TEMP'],
                mode='markers', 
                name='Temp Anomalies',
                marker=dict(color='red', size=8, symbol='x')
            ),
            row=1, col=1
        )
        fig.add_hline(y=450, line_dash="dash", line_color="red", row=1, col=1)
    
    # 2. RF 파워 시계열
    fig.add_trace(
        go.Scatter(
            x=filtered_df['timestamp'], 
            y=filtered_df['RF_POWER'],
            mode='lines', 
            name='RF Power',
            line=dict(color='green', width=2)
        ),
        row=1, col=2
    )
    
    # RF 파워 이상치 표시
    power_anomalies = filtered_df[filtered_df['RF_POWER'] > 300]
    if len(power_anomalies) > 0:
        fig.add_trace(
            go.Scatter(
                x=power_anomalies['timestamp'], 
                y=power_anomalies['RF_POWER'],
                mode='markers', 
                name='Power Anomalies',
                marker=dict(color='orange', size=8, symbol='triangle-up')
            ),
            row=1, col=2
        )
        fig.add_hline(y=300, line_dash="dash", line_color="orange", row=1, col=2)
    
    # 3. 온도 vs RF 파워 산점도
    fig.add_trace(
        go.Scatter(
            x=filtered_df['TEMP'], 
            y=filtered_df['RF_POWER'],
            mode='markers', 
            name='Temp vs Power',
            marker=dict(
                size=6, 
                opacity=0.6,
                color=filtered_df['CHAMBER_LOAD'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Chamber Load")
            )
        ),
        row=2, col=1
    )
    
    # 4. 데이터 요약 파이 차트
    equipment_counts = filtered_df['eqp_id'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=equipment_counts.index, 
            values=equipment_counts.values,
            name="Equipment Distribution"
        ),
        row=2, col=2
    )
    
    # 레이아웃 업데이트
    fig.update_layout(
        height=800,
        title_text=f"Manufacturing Data Analysis - {filter_str}",
        showlegend=True
    )
    
    # 축 라벨 추가
    fig.update_xaxes(title_text="Time", row=1, col=1)
    fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
    fig.update_xaxes(title_text="Time", row=1, col=2)
    fig.update_yaxes(title_text="RF Power", row=1, col=2)
    fig.update_xaxes(title_text="Temperature (°C)", row=2, col=1)
    fig.update_yaxes(title_text="RF Power", row=2, col=1)
    
    # 그래프 표시
    fig.show()
    
    # 통계 정보 출력
    print(f"\\n📊 필터링된 데이터 통계:")
    print(f"   데이터 포인트: {len(filtered_df):,}개")
    print(f"   온도 범위: {filtered_df['TEMP'].min():.1f} - {filtered_df['TEMP'].max():.1f}°C")
    print(f"   RF 파워 범위: {filtered_df['RF_POWER'].min():.1f} - {filtered_df['RF_POWER'].max():.1f}")
    print(f"   🔥 온도 이상치: {len(temp_anomalies)}개")
    print(f"   ⚡ RF 파워 이상치: {len(power_anomalies)}개")

# 함수 테스트
print("📈 시각화 함수 테스트:")
create_filtered_visualization('All', 'All', 'All')

📈 시각화 함수 테스트:


\n📊 필터링된 데이터 통계:
   데이터 포인트: 7,776개
   온도 범위: 417.6 - 488.1°C
   RF 파워 범위: 212.0 - 367.8
   🔥 온도 이상치: 77개
   ⚡ RF 파워 이상치: 76개


In [ ]:
# 인터랙티브 위젯과 시각화 함수 연결
if WIDGETS_AVAILABLE:
    print("🔗 인터랙티브 대시보드 연결 테스트:")
    
    # 방법 1: interact 함수 사용
    print("\\n1️⃣ interact() 함수 사용:")
    interactive_plot = interact(
        create_filtered_visualization,
        equipment_id=equipment_dropdown,
        chamber_id=chamber_dropdown,
        recipe=recipe_dropdown
    )
    
else:
    print("❌ ipywidgets를 사용할 수 없으므로 수동 테스트를 수행합니다.")
    
    # 수동 테스트를 위한 몇 가지 조합
    test_cases = [
        ('All', 'All', 'All'),
        ('EQP001', 'All', 'All'),
        ('All', 'A', 'All'),
        ('All', 'All', 'R101'),
        ('EQP001', 'A', 'R101')
    ]
    
    print("\\n🧪 수동 테스트 케이스:")
    for i, (eqp, chamber, recipe) in enumerate(test_cases, 1):
        print(f"\\n{i}. Equipment={eqp}, Chamber={chamber}, Recipe={recipe}")
        try:
            create_filtered_visualization(eqp, chamber, recipe)
        except Exception as e:
            print(f"❌ 테스트 실패: {e}")

🔗 인터랙티브 대시보드 연결 테스트:
\n1️⃣ interact() 함수 사용:


interactive(children=(Dropdown(description='Equipment:', index=2, layout=Layout(width='200px'), options=('All'…

## 5. 환경별 드랍다운 동작 테스트

각 환경(Jupyter/VS Code/터미널)에서 드랍다운이 정상적으로 동작하는지 확인하고, 문제가 있을 때 대체 방안을 제공합니다.

In [9]:
# 환경별 호환성 테스트
def test_widget_display():
    """위젯 표시 테스트 함수"""
    
    if not WIDGETS_AVAILABLE:
        print("❌ ipywidgets가 사용 불가능합니다.")
        return False
    
    try:
        # 간단한 테스트 위젯 생성
        test_widget = widgets.Dropdown(
            options=['Option 1', 'Option 2', 'Option 3'],
            value='Option 1',
            description='테스트:',
        )
        
        # 위젯 표시 시도
        display(test_widget)
        print("✅ 위젯 표시 성공")
        return True
        
    except Exception as e:
        print(f"❌ 위젯 표시 실패: {e}")
        return False

def create_enhanced_dashboard():
    """향상된 대시보드 (오류 처리 포함)"""
    
    print("🚀 향상된 인터랙티브 대시보드 생성...")
    
    if environment == "jupyter" and WIDGETS_AVAILABLE:
        try:
            # Jupyter 환경에서 위젯 사용
            print("📱 Jupyter 환경: ipywidgets 사용")
            
            # 위젯 생성
            eq_widget = widgets.Dropdown(options=equipment_ids, value='All', description='Equipment:')
            ch_widget = widgets.Dropdown(options=chamber_ids, value='All', description='Chamber:')
            rc_widget = widgets.Dropdown(options=recipes, value='All', description='Recipe:')
            
            # 위젯 표시
            controls = widgets.HBox([eq_widget, ch_widget, rc_widget])
            display(widgets.VBox([
                widgets.HTML("<h3>🎛️ 필터 선택</h3>"),
                controls
            ]))
            
            # 인터랙티브 연결
            def update_dashboard(equipment_id, chamber_id, recipe):
                clear_output(wait=True)
                display(widgets.VBox([
                    widgets.HTML("<h3>🎛️ 필터 선택</h3>"),
                    controls
                ]))
                create_filtered_visualization(equipment_id, chamber_id, recipe)
            
            # 위젯 변경 이벤트 연결
            interactive_dashboard = widgets.interactive(
                update_dashboard,
                equipment_id=eq_widget,
                chamber_id=ch_widget,
                recipe=rc_widget
            )
            
            display(interactive_dashboard)
            
        except Exception as e:
            print(f"❌ Jupyter 위젯 생성 실패: {e}")
            print("🔄 수동 인터페이스로 전환...")
            create_manual_interface()
            
    elif environment == "vscode":
        print("💻 VS Code 환경: 부분적 위젯 지원")
        
        try:
            # VS Code에서 위젯 테스트
            test_success = test_widget_display()
            
            if test_success:
                print("✅ VS Code에서 위젯 사용 가능")
                # 기본 인터랙티브 대시보드 사용
                interact(create_filtered_visualization,
                        equipment_id=equipment_ids,
                        chamber_id=chamber_ids,
                        recipe=recipes)
            else:
                print("⚠️ VS Code에서 위젯 표시 문제 발생")
                create_manual_interface()
                
        except Exception as e:
            print(f"❌ VS Code 테스트 실패: {e}")
            create_manual_interface()
            
    else:
        print("🖥️ 터미널 환경: 수동 인터페이스 사용")
        create_manual_interface()

def create_manual_interface():
    """수동 선택 인터페이스"""
    
    print("\\n=== 🖱️ 수동 필터 선택 인터페이스 ===")
    print("위젯을 사용할 수 없으므로 수동으로 선택하세요.")
    
    # 사전 정의된 시나리오
    scenarios = [
        ("전체 데이터", "All", "All", "All"),
        ("EQP001 전체", "EQP001", "All", "All"),
        ("Chamber A 전체", "All", "A", "All"),
        ("Recipe R101 전체", "All", "All", "R101"),
        ("EQP001 + Chamber A", "EQP001", "A", "All"),
        ("사용자 정의", "custom", "custom", "custom")
    ]
    
    print("\\n📋 미리 정의된 시나리오:")
    for i, (desc, eq, ch, rc) in enumerate(scenarios, 1):
        print(f"  {i}. {desc}")
    
    try:
        choice = int(input("\\n시나리오를 선택하세요 (1-6): "))
        
        if 1 <= choice <= 5:
            desc, eq, ch, rc = scenarios[choice-1]
            print(f"\\n선택된 시나리오: {desc}")
            create_filtered_visualization(eq, ch, rc)
            
        elif choice == 6:
            print("\\n사용자 정의 필터:")
            eq = input(f"Equipment {equipment_ids} [All]: ") or "All"
            ch = input(f"Chamber {chamber_ids} [All]: ") or "All"
            rc = input(f"Recipe {recipes} [All]: ") or "All"
            
            if eq in equipment_ids and ch in chamber_ids and rc in recipes:
                create_filtered_visualization(eq, ch, rc)
            else:
                print("❌ 잘못된 선택입니다.")
                
        else:
            print("❌ 잘못된 번호입니다.")
            
    except (ValueError, KeyboardInterrupt):
        print("\\n❌ 입력이 취소되거나 잘못되었습니다.")
        print("기본 전체 데이터 분석을 수행합니다.")
        create_filtered_visualization("All", "All", "All")

# 환경별 테스트 실행
print(f"\\n🧪 환경별 호환성 테스트 시작...")
print(f"현재 환경: {environment}")
print(f"위젯 사용 가능: {WIDGETS_AVAILABLE}")

create_enhanced_dashboard()

\n🧪 환경별 호환성 테스트 시작...


NameError: name 'environment' is not defined

## 💡 문제 해결 가이드

### 드랍다운이 보이지 않는 경우:

1. **패키지 설치 확인**:
   ```bash
   pip install ipywidgets
   conda install -c conda-forge ipywidgets
   ```

2. **Jupyter 확장 기능 활성화**:
   ```bash
   jupyter nbextension enable --py widgetsnbextension
   jupyter labextension install @jupyter-widgets/jupyterlab-manager
   ```

3. **VS Code 설정**:
   - Jupyter 확장 설치
   - Python 확장 최신 버전 사용
   - 노트북 신뢰(Trust) 설정 확인

4. **브라우저 문제**:
   - 페이지 새로고침 (Ctrl+F5)
   - 다른 브라우저에서 테스트
   - 브라우저 콘솔에서 JavaScript 오류 확인

### 환경별 권장사항:

- **Jupyter Notebook/Lab**: 완전한 위젯 지원
- **VS Code**: 부분적 지원, 때로는 수동 인터페이스 필요
- **Google Colab**: 위젯 지원하지만 제한적
- **터미널**: 수동 인터페이스만 사용

In [6]:
# 최종 테스트 및 요약
print("🏁 최종 테스트 및 요약")
print("="*50)

# 환경 정보 요약
print(f"📍 실행 환경: {environment}")
print(f"🔧 ipywidgets 사용 가능: {WIDGETS_AVAILABLE}")

# 데이터 요약
print(f"\\n📊 데이터 요약:")
print(f"   총 데이터 포인트: {len(df):,}개")
print(f"   Equipment 종류: {len(equipment_ids)-1}개")  # 'All' 제외
print(f"   Chamber 종류: {len(chamber_ids)-1}개")
print(f"   Recipe 종류: {len(recipes)-1}개")

# 이상치 요약
temp_anomalies = df[df['TEMP'] > 450]
power_anomalies = df[df['RF_POWER'] > 300]
print(f"\\n🚨 이상치 요약:")
print(f"   온도 이상치: {len(temp_anomalies)}개 ({len(temp_anomalies)/len(df)*100:.1f}%)")
print(f"   RF 파워 이상치: {len(power_anomalies)}개 ({len(power_anomalies)/len(df)*100:.1f}%)")

# 위젯 테스트 결과
if WIDGETS_AVAILABLE:
    try:
        # 마지막 위젯 상태 확인
        if 'equipment_dropdown' in locals():
            print(f"\\n🎛️ 위젯 상태:")
            print(f"   Equipment 선택: {equipment_dropdown.value}")
            print(f"   Chamber 선택: {chamber_dropdown.value}")
            print(f"   Recipe 선택: {recipe_dropdown.value}")
        
        print("\\n✅ 위젯 테스트 성공!")
        
    except Exception as e:
        print(f"\\n❌ 위젯 테스트 실패: {e}")
else:
    print("\\n⚠️ 위젯을 사용할 수 없어 수동 인터페이스를 사용했습니다.")

# 추천 사항
print(f"\\n💡 추천 사항:")
if environment == "jupyter":
    print("   - Jupyter 환경에서 최적의 성능을 보입니다.")
    print("   - 모든 인터랙티브 기능을 사용할 수 있습니다.")
elif environment == "vscode":
    print("   - VS Code에서는 제한적으로 동작할 수 있습니다.")
    print("   - 문제 발생 시 Jupyter Notebook 사용을 권장합니다.")
else:
    print("   - 터미널 환경에서는 수동 인터페이스를 사용하세요.")
    print("   - 더 나은 경험을 위해 Jupyter Notebook 사용을 권장합니다.")

print("\\n🎉 테스트 완료!")

🏁 최종 테스트 및 요약


NameError: name 'environment' is not defined